In [1]:
import os
from tqdm.notebook import tqdm

import openneuro

from ndspflow.workflows.workflow import WorkFlow

from neurodsp.filt import filter_signal
from neurodsp.spectral import compute_spectrum

from fooof import FOOOF, FOOOFGroup
from bycycle import Bycycle

# BIDS WorkFlow

Below, we fetch a BIDS dataset using the openneuro cli. Then define and execute the following workflow:

1. Load the BIDS directory using MNE BIDS
2. Apply custom functions to select channels and epoch the signal
3. Low-pass filter with neurodsp
4. Fit FOOOF
5. Fit Bycycle

In [2]:
%%capture
dataset = 'ds003844'
include = ['sub-RESP0059/ses-SITUATION1A', 'sub-RESP0280/ses-SITUATION1A']

# Create BIDs folder
bids_path = f'{os.getcwd()}/bids'

if not os.path.isdir(bids_path):
    os.makedirs(bids_path)
    
openneuro.download(dataset=dataset, target_dir=bids_path, include=include)

In [3]:
def select_channels(y_array, channels):
    """Select sub-set of channels."""
    inds = [ind for ind, ch in enumerate(channels)
            if ch in channels]
    
    return y_array[inds, :]
    
def epoch(y_array, n_epochs, epoch_len):
    """Epoch function."""
    y_array = y_array[:, :int(n_epochs) * epoch_len]
    y_array = y_array.reshape(-1, epoch_len)
    
    return y_array

In [4]:
wf = WorkFlow(bids_path=bids_path, session='SITUATION1A', task='acute')

wf.read_bids(subject='sub-RESP0059', allow_ragged=True)

# Custom transforms
wf.transform(select_channels, ['Gr16', 'Gr17'])
wf.transform(epoch, 2, 5000)
wf.transform(filter_signal, 'self.fs', 'lowpass', 200, remove_edges=False)

wf.fork(0)
wf.transform(compute_spectrum, 'self.fs')
wf.fit(FOOOF(verbose=False, max_n_peaks=5), (1, 100), axis=-1)

wf.fork(0)
wf.fit(Bycycle(), 'self.fs', (50, 100), axis=-1)

wf.run(n_jobs=1, progress=tqdm)

Running Workflow:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
wf.results

[[[<fooof.objs.fit.FOOOF at 0x7f522ab29d60>,
 [[<fooof.objs.fit.FOOOF at 0x7f522ab32ac0>,